In [1]:
## Learning while working LSTM models

# Cross Entropy Loss function works well with Label Encoder whereaas Binary Cross Entorpy is used for multi class classification.


# Transfer Learning and Multi Class Experiment 

In [3]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna

import time

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.utils.data import WeightedRandomSampler

c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Record Constants
INPUT_TRAIN_IN = "../../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../../data/train_out.csv"  
INPUT_TEST_IN = "../../../data/test_in.csv"
INPUT_TEST_OUT = "../../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


WINDOW_SIZE =  25 # Final RNA Sequence will be 101 Length 
MAX_LENGTH = (WINDOW_SIZE*2) +1

# 1 - One Hot Encoding with Pytorch in build Emnedding 
# 2 - K-mers with Word2Vec
# 3 - K-mers with Numerical Encoding
ENCODING_METHOD = 3


# 1 - LSTM with Cross Entropy 
MODEL = 1


# 1- Random Over Sampling
# 2 - Weighted Over Sampler 
SAMPLING_METHOD =1 

FRAMEWORK = "PYTORCH"


PERFORM_DATA_BALANCING = True
K_MERS_SIZE = 3
EMBEDDING_DIMENSION  = 3

# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'LABEL'


In [11]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )

x_data = pd.concat([x_train_raw, x_test_raw, x_valid_raw], axis=0, ignore_index=True)
y_data = pd.concat([y_train_raw, y_test_raw, y_valid_raw], axis=0, ignore_index=True)


middle_index = (x_train_raw.shape[1] // 2) + 1 # This is location for Modified Sequence . Use this as Y Target

if CROP_STRATEGY == 'MID':
    STRAT_INEDX =middle_index - WINDOW_SIZE -1 
    END_INDEX =middle_index + WINDOW_SIZE 

if CROP_STRATEGY == 'END':
    STRAT_INEDX =middle_index - (WINDOW_SIZE*2) -1 
    END_INDEX =middle_index-1 # Ignore Modified Position

x_data_cropped =  x_data.iloc[:,STRAT_INEDX :END_INDEX]  
concatenated_column= x_data_cropped.apply(lambda row: ''.join(map(str, row)), axis=1)
x_data_cropped = x_data_cropped.assign(Sequence=concatenated_column)

In [12]:
print(f" Shape of X : {x_data.shape} and  Shape of Y : {x_data.shape}")

 Shape of X : (309460, 1001) and  Shape of Y : (309460, 1001)


# Encode Y Category

In [13]:
## Filter Dataset to Keep only Target Binary Class 

RMs = ['hAm','hCm','hGm','hTm','hm1A','hm5C','hm5U','hm6A','hm6Am','hm7G','hPsi','Atol','NonMoD']
RMEncoding = [12,1,2,3,4,5,6,7,8,9,10,11,0]

def convert_y_to_original_labels(row):
    label = ""
    for index , n in enumerate(row.tolist()) :
        if n == 1 :
            label = RMs[index]
    if label == '':
        return 'NonMoD'
    return label

def get_original_y_lables( y_data ):
    # Convert One Hot Encoded Y to to Original Labels 
    y_original_labels = y_data.apply(convert_y_to_original_labels,axis=1)
    return y_original_labels  



### Encode X  - Vanilla One Hot Encoding

In [14]:
number_of_unique_kmers = set()
def encode_seq(kmer_token):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    number_of_unique_kmers.add(kmer_token)
    for  base in kmer_token:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

def applyOneHotEncoding(tokenized_sequences):
    encoded_sequences = []
    for seq in tokenized_sequences:
        encoded_sequences.append(encode_seq(seq)) 
 
    return np.array(encoded_sequences).flatten()


def encode_with_one_hot_encoding(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    concatenated_column= truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = truncated_df.assign(Sequence=concatenated_column)
    tokenized_sequences =  df_result['Sequence'].apply(applyOneHotEncoding).tolist()
    return tokenized_sequences


### K-mers with Word2Vec

In [8]:
# Model Documentation - 
# Encode RNA sequences using Word2Vec embeddings
import gensim

all_unique_k_mers = set()

def encode_sequence(sequence, word2vec_model):
    embeddings = []
    for kmer in sequence:
        embeddings.append(word2vec_model.wv[kmer])
    return np.array(embeddings)

def applyKmersAndEncoding(seq):
    k = 3
    tokenized_sequences = [''.join(seq[i:i+k]) for i in range(0, len(seq)-k+1)]
    return tokenized_sequences

def encode_x_with_word2Vec(x_train_raw):
    truncated_df =  x_train_raw.iloc[:,STRAT_INEDX :END_INDEX] # Window Starts from V501 with 50 window size
    truncated_df['Sequence'] = truncated_df.apply(lambda row: ''.join(map(str, row)), axis=1)
    tokenized_sequences =  truncated_df['Sequence'].apply(applyKmersAndEncoding).tolist()

    flat_list = [item for sublist in tokenized_sequences for item in sublist]
    all_unique_k_mers.update(flat_list)

    # The result, tokenized_sequences, is a list of lists, where each inner list
    #  contains the k-mers of the corresponding RNA sequence from the truncated_df list. 

    # Train Word2Vec model on tokenized_sequences (list of lists) instead of unique k-mers
    word2vec_model = gensim.models.Word2Vec(tokenized_sequences, vector_size=EMBEDDING_DIMENSION, window=5, min_count=1, workers=4)
    
    result = []
    for seq in tokenized_sequences:
        embedding = encode_sequence(seq, word2vec_model)
        result.append(embedding)
    return np.array(result)


### Enocde 3 Kmer with Numerical Encoding

In [15]:
kmer_dict = {}
encoding = 0
k = 3

for sequence in x_data_cropped['Sequence']:
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        if kmer not in kmer_dict:
            kmer_dict[kmer] = encoding
            encoding += 1

def encode_with_k_mer_codon(sequence):
    encoded_sequence = [] 
    for i in range(len(sequence) - k + 1):
        encoded_sequence.append(kmer_dict[sequence[i:i+k]] )
        
    return np.array(encoded_sequence)

def convertToTensor(x):
    return torch.tensor(x)

### Encode Y

In [16]:
def encode_target(y_data):
    if TARGET_ENCODING  == 'LABEL':
        le = LabelEncoder()
        y_encoded = le.fit_transform(y_data)
        #y_encoded = torch.tensor(y_encoded, dtype=torch.float32)

    if TARGET_ENCODING == 'ONE_HOT':
        ohe = OneHotEncoder(sparse=False)
        y_encoded = ohe.fit_transform(y_data.to_numpy().reshape(-1, 1))
        y_encoded = torch.tensor(y_encoded, dtype=torch.float32)

    return y_encoded

In [10]:
#test = np.array(encode_x_with_word2Vec(x_train_raw))
# print("Input Dimension : " , len(all_unique_k_mers))
# print("Shape of X  " , test[0][0])

## Prepare Data 

In [17]:
y_data_original =  get_original_y_lables(y_data)

if ENCODING_METHOD ==1:
    X_train = torch.tensor(encode_with_one_hot_encoding(x_train_raw) , dtype=torch.long)
    X_test = torch.tensor(encode_with_one_hot_encoding(x_test_raw) , dtype=torch.long)
    X_valid = torch.tensor(encode_with_one_hot_encoding(x_valid_raw) , dtype=torch.long)

if ENCODING_METHOD == 2:
    X_train = torch.tensor(encode_x_with_word2Vec(x_train_raw) , dtype=torch.float32)
    X_test = torch.tensor(encode_x_with_word2Vec(x_test_raw) , dtype=torch.float32)
    X_valid = torch.tensor(encode_x_with_word2Vec(x_valid_raw) , dtype=torch.float32)

if ENCODING_METHOD == 3:
   X_encoded  =  x_data_cropped['Sequence'].apply(encode_with_k_mer_codon)
   X_encoded = torch.tensor(X_encoded , dtype =  torch.long)


print("Encoding Completed for X Features.")
y_encoded = encode_target(y_data_original)


Encoding Completed for X Features.


C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_22348\629776211.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  X_encoded = torch.tensor(X_encoded , dtype =  torch.long)


In [21]:
y_data_original.value_counts()

NonMoD    154607
hm6A       65178
Atol       52618
hm1A       16341
hm5U        3696
hm5C        3207
hPsi        3137
hm6Am       2447
hTm         2253
hCm         1878
hAm         1591
hGm         1471
hm7G        1036
dtype: int64

In [22]:
# Generate Train and Test

print("Generate Train and Split..")
# Train set
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_encoded, test_size=0.3, random_state=42)

# Test and Validation set
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)



Generate Train and Split..


In [23]:

print("Encoding For Y Features.")
#Caste to Torch Tensor
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
y_valid = torch.tensor(y_valid, dtype=torch.long)


if SAMPLING_METHOD ==1 :
  print("Apply Random Over Sampling ")
  sm = RandomOverSampler(random_state=42)
  X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

if SAMPLING_METHOD ==2 :
  print("Apply Weighted Over Sampling ")
  # # Calculate class weights
  class_weights = 1.0 / torch.bincount(y_train)
  weights = class_weights[y_train]
  sampler = WeightedRandomSampler(weights, len(weights))

Encoding For Y Features.
Apply Random Over Sampling 


In [32]:
y_resampled = torch.tensor(y_resampled, dtype=torch.long)
counts = torch.bincount(y_resampled)
for i, count in enumerate(counts):
    print(f"Value: {i}, Count: {count}")

X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
X_valid = X_valid.to(torch.float32)
X_train.dtype

Value: 0, Count: 108371
Value: 1, Count: 108371
Value: 2, Count: 108371
Value: 3, Count: 108371
Value: 4, Count: 108371
Value: 5, Count: 108371
Value: 6, Count: 108371
Value: 7, Count: 108371
Value: 8, Count: 108371
Value: 9, Count: 108371
Value: 10, Count: 108371
Value: 11, Count: 108371
Value: 12, Count: 108371


C:\Users\shashi.vish\AppData\Local\Temp\ipykernel_22348\2993319789.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_resampled = torch.tensor(y_resampled, dtype=torch.long)


torch.float32

### Hyper Parameters

In [13]:
# Destroy Objects for Memory Efficiency

x_data = None
y_data = None
y_valid_raw = None
x_valid_raw = None
x_train_raw = None
y_train_raw = None
x_test_raw = None
y_test_raw = None
y_data_original = None

In [33]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_dim):
        super(BahdanauAttention, self).__init__()
        self.W = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1)

    def forward(self, hidden, encoder_outputs):
        hidden = hidden.unsqueeze(1)
        hidden = hidden.expand(-1, encoder_outputs.size(1), -1)
        alignment_scores = self.v(torch.tanh(self.W(hidden + encoder_outputs)))
        attention_weights = F.softmax(alignment_scores, dim=1)
        context_vector = torch.sum(attention_weights * encoder_outputs, dim=1)
        return context_vector, attention_weights

class RNATransferLearning(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim, num_layers, output_dim, dropout=0.5):
        super(RNATransferLearning, self).__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.attention = BahdanauAttention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim) 
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.long()
        x_embeded = self.embedding(x)

        lstm_out, (h, c) = self.lstm(x_embeded)
        
        hidden = h.squeeze(0)

        #apply attention using the last hidden state and the encoder outputs
        context_vector, attention_weights = self.attention(hidden, lstm_out)
        
        #dropout
        output = self.dropout(context_vector)

        #final output
        out = self.fc(output)
        
        return out


In [35]:
import torch
import torch.nn as nn

model_name = 'rna_pre_trained_model.pt'
#Model Parameters  :  {'INPUT_DIMENSION': 64, 
# 'HIDDEN_DIMENSION': 32, 
# 'NO_OF_LAYERS': 1, 
# 'BATCH_SIZE': 32,
#  'OUTPUT_DIMENSION': 4, 
# 'EMBEDDING_DIMENSION': 7,
#  'DROP_OUT': 0.1}

# Define hyperparameters
hyperparameters = {
    'INPUT_DIMENSION': 4,
    'HIDDEN_DIMENSION': 3,
    'NO_OF_LAYERS': 1,
    'BATCH_SIZE': 32,
    'OUTPUT_DIMENSION': 4,
    'EMBEDDING_DIMENSION': 7,
    'DROP_OUT': 0.1
}

# Instantiate the pretrained model
pretrained_model = RNATransferLearning(
    input_dim=hyperparameters['INPUT_DIMENSION'],
    embedding_dim=hyperparameters['EMBEDDING_DIMENSION'],
    hidden_dim=hyperparameters['HIDDEN_DIMENSION'],
    num_layers=hyperparameters['NO_OF_LAYERS'],
    output_dim=hyperparameters['OUTPUT_DIMENSION'],
    dropout=hyperparameters['DROP_OUT']
)

# Load the saved state_dict into the model
pretrained_model.load_state_dict(torch.load(model_name))

# Define the new output dimension
new_output_dim = 13

# Define a new model that extends the pretrained model
class RNAWithPreTrainedModel(nn.Module):
    def __init__(self, pretrained_model):
        super(RNAWithPreTrainedModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.fc = nn.Linear(pretrained_model.fc.out_features, new_output_dim)

    def forward(self, x):
        features = self.pretrained_model(x)
        output = self.fc(features)
        return output

# Create an instance of the new model
model = RNAWithPreTrainedModel(pretrained_model)

print(model)
print(hyperparameters)

RuntimeError: Error(s) in loading state_dict for RNATransferLearning:
	Missing key(s) in state_dict: "attention.W.weight", "attention.W.bias", "attention.v.weight", "attention.v.bias". 
	size mismatch for fc.weight: copying a param with shape torch.Size([4, 3]) from checkpoint, the shape in current model is torch.Size([13, 3]).
	size mismatch for fc.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([13]).

In [36]:
class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    
def train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, num_epochs):
    best_val_loss = float('inf')
    consecutive_no_improvement = 0
   
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0

        for inputs, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Compute validation loss and accuracy
        val_loss, val_accuracy ,true_labels , pred_labels = evaluate_model(model, val_dataloader, loss_function)

        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            consecutive_no_improvement = 0
        else:
            consecutive_no_improvement += 1

        # Break training if there is no improvement for three consecutive epochs
        if consecutive_no_improvement == 3:
            print("No improvement for three consecutive epochs. Stopping training.")
            break

        end_time = time.time()
        total_time = end_time - start_time
        
        print(f"Epoch {epoch + 1}, Train Loss: {running_loss / len(train_dataloader):.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f} , Time Taken :  {total_time:.2f}")



def evaluate_model(model, dataloader, loss_function):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    true_labels = []
    pred_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            true_labels.extend(labels.tolist())
            pred_labels.extend(predicted.tolist())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = running_loss / len(dataloader)
    accuracy = correct / total
    return loss, accuracy , true_labels , pred_labels


In [24]:

print("Started Model Training..")

# Create dataset and dataloader
train_dataset = RNADataset(X_train, y_train)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameters['BATCH_SIZE'], sampler=sampler)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameters['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameters['BATCH_SIZE'], shuffle=False)


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# GPU uncomment below line for parallel computation.
#model = nn.DataParallel(model)

print(model)
print("Model Parameters  : " , hyperparameters)

train_model(model, train_dataloader, test_dataloader, optimizer, loss_function, num_epochs)


Started Model Training..
Total Number of Parameters for Model Training : 406 
RNAWithPreTrainedModel(
  (pretrained_model): RNATransferLearning(
    (embedding): Embedding(4, 7)
    (lstm): LSTM(7, 3, batch_first=True, dropout=0.5)
    (fc): Linear(in_features=3, out_features=13, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=13, out_features=13, bias=True)
)
Model Parameters  :  {}
Epoch 1, Train Loss: 1.9121, Val Loss: 2.1309, Val Accuracy: 0.1667 , Time Taken :  54.48
Epoch 2, Train Loss: 1.8558, Val Loss: 2.1624, Val Accuracy: 0.1700 , Time Taken :  64.02
Epoch 3, Train Loss: 1.8178, Val Loss: 2.4497, Val Accuracy: 0.1667 , Time Taken :  61.22
No improvement for three consecutive epochs. Stopping training.


### Calculate Model Accuracy

In [25]:
# Evaluate the model on the test dataset
val_loss, val_accuracy ,true_labels , pred_labels = evaluate_model(model, valid_dataloader, loss_function)

# Print the final accuracy
print(f"Final Accuracy: {val_accuracy:.4f}")

# Print the classification summary
print("\n Classification Summary:")
print(classification_report(true_labels, pred_labels))

Final Accuracy: 0.2226

 Classification Summary:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       150
           1       0.48      0.04      0.08      1799
           2       0.00      0.00      0.00       150
           3       0.25      1.00      0.40       150
           4       0.25      1.00      0.40       150
           5       0.00      0.00      0.00       150
           6       0.00      0.00      0.00       150
           7       0.00      0.00      0.00       150
           8       0.00      0.00      0.00       150
           9       0.17      1.00      0.29       150
          10       0.15      0.81      0.26       150
          11       0.28      1.00      0.44       150
          12       0.00      0.00      0.00       150

    accuracy                           0.22      3599
   macro avg       0.12      0.37      0.14      3599
weighted avg       0.29      0.22      0.11      3599



c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

### Optuna Hyperparameter Training

In [ ]:
import optuna

def objective(trial):
    # Suggest hyperparameters
    embedding_dim = trial.suggest_int("embedding_dim", 16, 300)
    hidden_dim = trial.suggest_int("hidden_dim", 32, 512)
    num_layers = trial.suggest_int("num_layers", 1, 4)
    num_epochs = 100
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_categorical("batch_size", [256, 512, 1024])

    # Create dataset and dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss function, and optimizer
    model = RNAClassifier(hyperparameter['INPUT_DIMENSION'], embedding_dim, hidden_dim, num_layers, hyperparameter['OUTPUT_DIMENSION'], dropout)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # GPU uncomment below line for parallel computation.
    model = nn.DataParallel(model)
    train_model(model, train_dataloader, test_dataloader, device, num_epochs, optimizer, loss_function)

    # Evaluate the model
    _, val_acc, _, _ = validate_model(model, valid_dataloader, device, loss_function)

    return val_acc

# Create an Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(" Value: {:.4f}".format(trial.value))
print(" Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


### LSTM Model with Binary Cross Entropy Loss (Works with One Hot Encoder)

In [ ]:
# ## Define LSTM and Train

# class RNADataset(Dataset):
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, index):
#         return self.X[index], self.y[index]

# class RNAClassifier_BCE(nn.Module):
#     def __init__(self, unqique_k_mer_size, embedding_dim,hidden_dim,num_layers, output_dim):
#         super(RNAClassifier_BCE, self).__init__()
#         self.embedding  = nn.Embedding(unqique_k_mer_size,embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):

#         x_embeded = self.embedding(x)
#         _, (hidden, _) = self.lstm(x_embeded)

#        # print("Hidden Dimansion : " , hidden.shape)
#         x = self.fc(hidden[-1]) # Squeeze does not make any differecent in Tensor , basically removing 0 demsion from tensor 

#         # When we squeeze a tensor, the dimensions of size 1 are removed. 
#         # The elements of the original tensor are arranged with the remaining dimensions. For example, 
#         # if the input tensor is of shape: (m×1×n×1) then the output tensor after squeeze will be of shape: (m×n). 

#         # print("Ouput  Dimansion : " ,x.shape)      

#         # print("Result After Applygin Softmax : " , torch.softmax(x, dim=-1) )                                      
#         return x



# if MODEL==1 :
#     print("Excuting LSTM Model")

#     # Create dataset and dataloader
#     print("X Shape of Tensor : " , X_train.shape , " Y Tensor Shape : " , y_train.shape)
#     print("HyperParameter for Training : ", hyperparameter)
#     train_dataset = RNADataset(X_train, y_train)
#     test_dataset = RNADataset(X_test, y_test)

#     train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#     test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

#     # Calculating INPUT_DIMENSION for LSTM Mode 
#     # If you are just using One Hot then INPUT_DIMENSION would be 4 because there are 4 possible unique nucleosides
#     # But if you are using K mers then INPUT_DIMENSION would be number of unique k-mers on all dataset

#     # LSTM model Format - (batch_size, sequence_length, input_size)  
#     # batch_size comes from DataLoaded , you can tune it.
#     # Sequence Length - Lenth of X[0] shape (32, 19, 12)  Here 19 is sequence length , 
#     # Input Size = 12 length of X[0][0] - Basically all unique nucleoside or k-mers

#     # Initialize the model, loss function, and optimizer
#     # model = RNAClassifier_BCE(input_size=4788, 
#     #                           hidden_size=hyperparameter['HIDDEN_DIMENSION'] , 
#     #                           num_layers = hyperparameter['NO_OF_LAYERS'],
#     #                           output_size=hyperparameter['OUTPUT_DIMENSION'] )
#     model = RNAClassifier_BCE(unqique_k_mer_size=hyperparameter['INPUT_DIMENSION'],
#                               embedding_dim=hyperparameter['EMBEDDING_DIMENSION'] 
#                               hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
#                               num_layers = hyperparameter['NO_OF_LAYERS'],
#                               output_dim=hyperparameter['OUTPUT_DIMENSION'] )
    
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters())

#     # Number of Parameters for Model
#     total_parameters = []
#     for p in model.parameters():
#         total_parameters.append(p.numel())
    
#     print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

#     # Train the model
#     num_epochs = 10
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
    
#     for epoch in range(num_epochs):
#         model.train()
#         running_loss = 0.0
#         correct = 0
#         total = 0

#         for i, (inputs, labels) in enumerate(train_dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)

#             print("Target Labesl : " , labels.shape)
#             print("Ouput Labesl : " , outputs.shape)

#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
            
#         epoch_loss = running_loss / len(train_dataloader)
#         print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")




In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        outputs = model(x_batch)
        
        # Basically models predict probability for each class and then finally we pick up with hightest probabiliyt class 
        # and then compare with actual class to calculate accuracy. 
        
        ## Predicted torch.max(outputs, 1) 
#         values=tensor([0.7304, 0.4554, 0.3984, 0.4091, 0.9271, 0.4370, 0.3837, 0.4395, 0.4005,
#         0.5451, 0.5642, 0.6866, 0.5768, 0.5160, 0.5093, 0.4947, 0.6823, 0.4612,
#         0.4073, 0.5041, 0.6982, 0.4116, 0.4092, 0.5589, 0.6010, 0.4693, 0.6324,
#         0.5524, 0.4339, 0.7889, 0.5216, 0.6588]),
#         indices=tensor([ 1, 10, 10, 10,  1, 10, 10, 10,  1, 10,  1,  1,  0,  1,  1, 10,  1, 10,
#         10, 10, 10,  1, 10,  0, 10,  1,  1,  0, 10,  1,  1,  0]))

        _, predicted = torch.max(outputs, 1)  # Get the class index with the highest probability
        #print(torch.max(outputs, 1))
        true_labels = torch.max(y_batch, 1).indices  # Get the true class indices from the one-hot encoded labels [0,1,0,0,,,... ] Get index of 1
        correct_predictions += (predicted == true_labels).sum().item()
        total_predictions += y_batch.size(0)

accuracy = correct_predictions / total_predictions


print(f"Validation Accuracy: {accuracy:.4f}")
